In [3]:
import docIterators
import pymongo
from pymongo import MongoClient
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import collections
import codecs
import json
import random
from bhtsne.bhtsne import TSNE as BHTSNE
import sys
import analysis_tools as at

In [4]:
conn=['mongodb://localhost:27017','Cherry','Passionfruit']
#iterator = docIterators.StrawberryIter(conn,iter_type='VECTORS')
#mc = MongoClient(conn[0])[conn[1]][conn[2]]

In [40]:
with codecs.open('journals.json','r',encoding='utf8') as f:
    c=json.load(f)
c=collections.Counter(c)
vector_type='d2v-d'
journo_coll={}
for journal in zip(*c.most_common()[:20])[0]:
    journo_vectors=[]
    curs = mc.find({'journal':journal})
    for rec in curs[:1000]:
        vectors = rec['vectors']
        journo_vectors.append(vectors[vector_type])
    journo_coll[journal]=journo_vectors
    print(journal + ' collected')

Chemistry - A European Journalcollected
Chem. Lett.collected
Phys. Chem. Chem. Phys.collected
Journal of Materials Chemistrycollected
Food Chemistrycollected
Bioorganic & Medicinal Chemistry Letterscollected
Journal of Heterocyclic Chemistrycollected
Chemistry of Heterocyclic Compoundscollected
Int. J. Quantum Chem.collected
ChemInformcollected
J. Polym. Sci. A Polym. Chem.collected
Journal of Radioanalytical and Nuclear Chemistrycollected
European Journal of Organic Chemistrycollected
Bioorganic & Medicinal Chemistrycollected
Anal Bioanal Chemcollected
Eur. J. Inorg. Chem.collected
Journal of Solid State Chemistrycollected
Organic & Biomolecular Chemistrycollected
Environmental Toxicology and Chemistrycollected
Materials Chemistry and Physicscollected


In [42]:
vecs= [a for b in journo_coll.values() for a in b]
#t = TSNE(init='pca',n_components=2)
t=BHTSNE(n_components=2)
reduced_vecs = t.fit_transform(vecs)
colors=iter(plt.cm.rainbow(np.linspace(0,1,len(journo_coll.keys()))))
start_ind=0
for j in journo_coll.keys():
    no_to_plot=len(journo_coll[j])
    end_ind=start_ind+no_to_plot
    color=colors.next()
    plt.plot(reduced_vecs[start_ind:end_ind,0], reduced_vecs[start_ind:end_ind,1], '.',c=color)
    start_ind=end_ind
    plt.show()

In [13]:
vecs= []
ind=0
for record in iterator:
    vecs.append(record['vectors']['d2v-d'])
    if ind==50000:
        break
    ind+=1
#t = TSNE(init='pca',n_components=2)
t=BHTSNE(n_components=2)
reduced_vecs = t.fit_transform(vecs)
plt.plot(reduced_vecs[:,0], reduced_vecs[:,1],'o')
plt.show()

[#         ] 10% 50000bhtsne/bh_tsne


KeyboardInterrupt: 

In [5]:
iterator = docIterators.StrawberryIter(conn,iter_type='VECTORS',query={'source_url':{'$regex':'www.ch.cam.ac.uk'}})


In [6]:
vec,dois=at.get_vectors(iterator)

[######### ] 95% 9000



In [7]:
mat = at.cosine_mat(vec,vec)

In [8]:
def csv_gephi_exporter(dois,mat,filename='cam_connect.csv',thresh=0.35):
    with open(filename,'w') as f:
        masked=mat*(mat>thresh)
        f.write(';'+';'.join(dois)+'\n')
        for i in range(mat.shape[0]):
            masked[i,i]=0.
            ex=(';'.join([str(j) for j in masked[i] if j>thresh]))
            ex=dois[i]+';'+ex+'\n'
            f.write(ex)


In [35]:
def gexf_gephi_exporter(dois,mat,filename='cam_connect.gexf',thresh=.35):
    node_count=len(dois)
    edge_count=(np.sum(mat>thresh)-mat.shape[0])/2
    header='''<?xml version="1.0" encoding="UTF-8"?>
<gexf xmlns:viz="http:///www.gexf.net/1.1draft/viz" version="1.1" xmlns="http://www.gexf.net/1.1draft">
<meta lastmodifieddate="2010-03-03+23:44">
<creator>Gephi 0.7</creator>
</meta>
<graph defaultedgetype="undirected" idtype="string" type="static">
<nodes count="'''
    header=header+str(node_count)+'">'
    footer='''\n</edges>\n</graph>\n</gexf>'''
    with open(filename,'w') as f:
        f.write(header)
        for i in range(node_count):
            node='\n<node id="'+str(float(i))+'" label="'+dois[i]+'"/>'
            f.write(node)
        f.write('\n</nodes>\n<edges count="'+str(edge_count)+'">')
        edge_no=0
        for i in range(node_count):
            for j in range(i+1,node_count):
                if mat[i,j]>thresh:
                    f.write(
                        '\n<edge id="'+
                        str(float(edge_no))+
                        '" source="'+str(float(i))
                        +'" target="'+str(float(j))+
                        '" weight="'+str(mat[i,j])+'"/>')
                    edge_no+=1
        f.write(footer)

In [36]:
gexf_gephi_exporter(dois,mat)

In [233]:
counts, edges = np.histogram(mat,bins=100)

In [249]:
plt.plot(centres,list(counts))
plt.show()

In [239]:
plt.show()

In [242]:
edges

array([-0.53240638, -0.51708857, -0.50177076, -0.48645295, -0.47113514,
       -0.45581733, -0.44049952, -0.42518171, -0.4098639 , -0.39454609,
       -0.37922828, -0.36391047, -0.34859266, -0.33327485, -0.31795704,
       -0.30263923, -0.28732142, -0.27200361, -0.2566858 , -0.24136799,
       -0.22605018, -0.21073237, -0.19541456, -0.18009675, -0.16477894,
       -0.14946113, -0.13414332, -0.11882551, -0.1035077 , -0.08818989,
       -0.07287208, -0.05755427, -0.04223646, -0.02691865, -0.01160084,
        0.00371697,  0.01903478,  0.03435259,  0.0496704 ,  0.0649882 ,
        0.08030601,  0.09562382,  0.11094163,  0.12625944,  0.14157725,
        0.15689506,  0.17221287,  0.18753068,  0.20284849,  0.2181663 ,
        0.23348411,  0.24880192,  0.26411973,  0.27943754,  0.29475535,
        0.31007316,  0.32539097,  0.34070878,  0.35602659,  0.3713444 ,
        0.38666221,  0.40198002,  0.41729783,  0.43261564,  0.44793345,
        0.46325126,  0.47856907,  0.49388688,  0.50920469,  0.52

In [245]:
centres=[]
for i in range(100):
    centres.append((edges[i]+edges[i+1])/2.)

In [247]:
centres

[-0.5247474728054029,
 -0.50942966299296388,
 -0.49411185318052497,
 -0.47879404336808595,
 -0.46347623355564699,
 -0.44815842374320802,
 -0.43284061393076906,
 -0.41752280411833009,
 -0.40220499430589107,
 -0.38688718449345216,
 -0.37156937468101314,
 -0.35625156486857423,
 -0.34093375505613521,
 -0.32561594524369625,
 -0.31029813543125728,
 -0.29498032561881832,
 -0.27966251580637935,
 -0.26434470599394033,
 -0.24902689618150139,
 -0.23370908636906243,
 -0.21839127655662346,
 -0.2030734667441845,
 -0.18775565693174553,
 -0.17243784711930657,
 -0.1571200373068676,
 -0.14180222749442864,
 -0.12648441768198965,
 -0.11116660786955065,
 -0.095848798057111689,
 -0.080530988244672724,
 -0.065213178432233759,
 -0.049895368619794794,
 -0.034577558807355829,
 -0.019259748994916837,
 -0.0039419391824778716,
 0.011375870629961093,
 0.026693680442400058,
 0.042011490254839023,
 0.057329300067277988,
 0.072647109879716953,
 0.087964919692155918,
 0.10328272950459488,
 0.1186005393170339,
 0.133918

In [268]:
sum(list(counts[54:]))/2

466527

In [267]:
centres[54]

0.30241425706630154

In [14]:
thresh=0.35
ind=0
for i in range(mat.shape[0]):
    for j in range(i+1,mat.shape[0]):
        if mat[i,j]>thresh:
            ind+=1
print ind
np.sum(mat>thresh)

160779


331025

In [19]:
np.array([[1,3,4],[1,1,5]])*(np.array([[1,3,4],[1,1,5]])>2)

array([[0, 3, 4],
       [0, 0, 5]])

In [16]:
(np.sum(mat>thresh)-mat.shape[0])/2

160779

In [17]:
mat[1,1]

0.99999999999999978

In [26]:
len(dois)

9467

In [27]:
mat.shape

(9467, 9467)